<a href="https://colab.research.google.com/github/TheOneTrueGuy/DigitalDayDream/blob/main/1simple_SDXL_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade diffusers transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.4 MB/s eta 0:00:00


In [2]:
#@title You gotta use this too if you want easy save to you google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# run this first then make anim in next cell
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline
from diffusers.utils import load_image
#from diffusers import DiffusionPipeline
import datetime as dt
import time
import os
import random
from PIL import ImageFont, ImageDraw, Image

class FakeSafety():
    def __call__(self, clip_input, images):
        return (images, False)


# check to see if pipe is loaded, if not load it otherwise skip loading pipe

#if pipe is None:



#pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
#  "stabilityai/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16
#) # can't use LoRa??

pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
  "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16
)


#chillpixel/starlight-animated-sdxl

pipe.load_lora_weights("goofyai/cyborg_style_xl") # use with "cyborg" or "android" keyword
#pipe.load_lora_weights("datamonet/Makima_Chainsaw_Man_LoRA") #needs weight_name
#pipe.load_lora_weights("") #spare for swapping in new loras

# cadaeic/duelyst-landscape-lora
# Duskfallcrew/the-crystal-exarch-15
# datamonet/Makima_Chainsaw_Man_LoRA


pipe = pipe.to("cuda")
pipe.safety_checker = FakeSafety()

# the following might work with some param tweaking but I didn't have good luck with these LoRas
# Norod78/sdxl-pumpkin-head-lora #not good
# lora-library/tekakutli-dinosaurs #didn't work


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [17]:
#@title upload start image 512x512 will be quick and look good

import os

from google.colab import files
uploaded = files.upload()
name=uploaded.keys()
name = list(name)[0]
print(name)


Saving st-0011.png to st-0011.png
st-0011.png


st-0040.png


In [ ]:

zip_name = "easy_to_remember_name.zip" #@param {type:"string"}
negprompt = " monochrome, dumb, flat, boring, ordinary" #@param {type:"string"}
prompt= "Something really amazing that everyonoe wants to see" #@param {type:"string"}


outdir = dt.datetime.now().strftime("%Y%m%d%H%M")
# if outdir exists add 4 random digits to it:
if os.path.exists(outdir):
    # Generate 4 random digits
    random_digits = str(random.randint(1000, 9999))

    # Create a new output directory with the random digits appended
    new_outdir = outdir + random_digits

    os.mkdir(new_outdir)
    outdir=new_outdir
    print("Created new output directory:", new_outdir)
else:
    os.system(f"mkdir {outdir}")
print("date time now:" + outdir)

xz=0


total=100

for xn in range(0,total,1):
  if xn%50==49: torch.cuda.empty_cache()
  init_image = load_image(name).convert("RGB")#

  image = pipe(
               prompt,
               num_inference_steps=14,
               strength=0.81,
               guidance_scale=7.34,
               negative_prompt = negprompt ,
               image=init_image
               ).images[0]
  outname= name

  out2=outdir+"/st-" + str(xz).zfill(4) + ".png"
  image.save(out2) # save animation frame
  xz+=1
  image.save(outname)
  #next 2 lines apply motion distortion from moto*.txt motion files
  # deleted lines
os.system(f"zip -r {zip_name} {outdir}")
os.system(f"cp {zip_name}  /content/drive/MyDrive")
